In [1]:
%load_ext autoreload
%env SPARK_HOME=/usr/hdp/current/spark2-client

import findspark
findspark.init()

print('findspark initialized ...')

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, column, max, min

spark = SparkSession.builder.appName('mlonspark')\
    .config('spark.executor.instances', '7')\
    .getOrCreate()

print('pyspark ready ...')

env: SPARK_HOME=/usr/hdp/current/spark2-client
findspark initialized ...
pyspark ready ...


In [2]:
train = spark.read.load("/data/lastfm-dataset-360K/data-filtered-std-pos-train.parquet")
test = spark.read.load("/data/lastfm-dataset-360K/data-filtered-std-pos-test.parquet")

In [3]:
from pyspark.ml.recommendation import ALS
alg = ALS()\
    .setUserCol("userId")\
    .setItemCol("artistId")\
    .setRatingCol("stdCountPos")\
    .setRank(25)\
    .setRegParam(0.0)\
    .setAlpha(10.0)\
    .setImplicitPrefs(True)

model = alg.fit(train)

IllegalArgumentException: 'Field "artistId" does not exist.\nAvailable fields: userHash, artistName, userId, users-userHash, artists-artistId, artists-artistName, listenCount, stdCount, stdCountPos'

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import isnan
evaluator = RegressionEvaluator()\
    .setMetricName("rmse")\
    .setLabelCol("stdCountPos")\
    .setPredictionCol("prediction")

trainPredictions = model.transform(train)
trainPredictionsFiltered = trainPredictions.where(~isnan(col("prediction")))  
trainRmse = evaluator.evaluate(trainPredictionsFiltered)

testPredictions = model.transform(test)
testPredictionsFiltered = testPredictions.where(~isnan(col("prediction")))    
testRmse = evaluator.evaluate(testPredictionsFiltered)

print("train RMSE = %f" % trainRmse)
print("test RMSE = %f" % testRmse)